# Indentificando fraude a partir de e-mails da Enron




## Enron Corporation

>A Enron Corporation foi uma companhia de energia americana, localizada em Houston, Texas. Empregava cerca de 21 000 pessoas, tendo sido uma das empresas líderes no mundo em distribuição de energia (electricidade, gás natural) e comunicações, mas decretou falência. Seu faturamento atingia 101 bilhões de dólares em 2000, pouco antes do escândalo financeiro que ocasionou sua falência.

>Alvo de diversas denúncias de fraudes contábeis e fiscais e com uma dívida de 13 bilhões de dólares, o grupo pediu concordata em dezembro de 2001 e arrastou consigo a Arthur Andersen, que fazia a sua auditoria. Na época, as investigações revelaram que a Enron havia manipulado seus balanços , com a ajuda de empresas e bancos, e escondera dívidas de 25 bilhões de dólares por dois anos consecutivos, tendo inflado artificialmente os seus lucros.

>O governo dos Estados Unidos abriu dezenas de investigações criminais contra executivos da Enron e da Arthur Andersen. A Enron foi também processada pelas pessoas lesadas. De acordo com os investigadores, os executivos e contadores, assim como instituições financeiras e escritórios de advocacia, que à época trabalhavam para a companhia, foram, de alguma forma e em diferentes graus, responsáveis pelo colapso da empresa.
>Em razão de uma série de escândalos financeiros corporativos, como o da Enron, foi redigida a lei Sarbanes-Oxley, em 2002.

>O ocorrido serviu de palco para o filme de comédia As Loucuras de Dick & Jane (2005) Com Jim Carrey, onde ele trabalha na Globodyne (Fictícia Enron) e passa altos perrengues devido a falência da empresa. Vale notar um EasterEgg no final, onde seu vizinho ostentador se gaba por ter conseguido um emprego na incrível Enron.

[wikipedia](https://pt.wikipedia.org/wiki/Enron)







## Visão Geral do Projeto

Neste projeto, irei bancar o detetive, e construirei um modelo de machine learning que visará determinar se um funcionário é ou não uma Person Of Interest  (POI). Um POI é um funcionário que participou do escândalo da Enron. Os dados são uma combinação de dados financeiros e dados de e-mails dos funcionários investigados neste caso de fraude, o que significa que eles foram indiciados, fecharam acordos com o governo, ou testemunharam em troca de imunidade no processo.


### Processo de construção e treinamento do modelo

A construção do modelo preditivo seguirá um procecesso geral com as seguintes etapas:
   1. **Explorar os dados:** Nessa etapa será realizada uma breve análise exploratória com os passos a seguir:
           1. Estudar as características de cada atributo.
           2. Identificar valores faltantes.
           3. Identificar ruídos nos dados(outliers)
           4. Estudar o tipo da distribuição.
           5. Estudar possíveis correlações entre os atributos.
   2. **Preparar os dados:**
           1. Seleção de características(atributos)
           2. Fixar ou remover ruídos.
           3. Preencher valores ausentes(com média, zero...) ou removê-los.
           4. Engenharia de características
   3. **Escolha e Afinamento de um Algorítmo:**
           1. .... 
   4. **Avaliar e Validar:**
           1. ...